This notebook is part of the PyImageJ [Tutorial Series](./notebooks.rst), and assumes familiarity with the ImageJ API. Dedicated tutorials for ImageJ can be found [here](https://imagej.net/tutorials/).

# 5 Convenience methods of PyImageJ

PyImageJ is built to provide easy access to key ImageJ resources. We call these collective methods "convenience methods". These methods are attached to`ij.py` after initializing ImageJ. Here's a quick list of some of the more useful methods and additional information.

| `ij.py.` | function | more information |
| :---: | :---: | :---: |
| `show` | Show an image | [06-Working-with-Images](06-Working-with-Images.ipynb)
| `to_java` | Convert data from Python to Java | [03-Sending-Data-to-Java](03-Sending-Data-to-Java.ipynb) |
| `from_java` | Convert data from Java to Python | [04-Retrieving-Data-from-Java](04-Retrieving-Data-from-Java.ipynb) |
| `run_macro` | Run an original ImageJ macro | [07-Running-Macros-Scripts-and-Plugins](07-Running-Macros-Scripts-and-Plugins.ipynb) |
| `run_script` | Run an ImageJ script (supported languages) | [07-Running-Macros-Scripts-and-Plugins](07-Running-Macros-Scripts-and-Plugins.ipynb) |
| `run_plugin` | Run a plugin | [07-Running-Macros-Scripts-and-Plugins](07-Running-Macros-Scripts-and-Plugins.ipynb) |
| `initialize_numpy_image` | Create a new numpy image in the same shape as input image | [06-Working-with-Images](06-Working-with-Images.ipynb) |
| `sync_image` | Synchronize data between ImageJ and ImageJ2 data structures | -- |
| `active_dataset` | Get the active image as a `Dataset` | -- |
| `active_xarray` | Get a copy of the active image as an `xarray.DataArray` | -- |
| `active_imageplus` | Get the `ImagePlus` from the current window | -- |

There are other convenience methods that are attached to `ij.py`. After initializing ImageJ you can explore `ij.py`'s methods with `dir`.

## 5.1 Other convenient access to ImageJ functions

When the original ImageJ is available (_i.e._ the legacy layer is active) `IJ`, `WindowManager`, `ResultsTable` and `RoiManager` are accessible directly from the initialized `ij` object.

In [1]:
import imagej

# initialize imagej
ij = imagej.init()
print(f"ImageJ2 version: {ij.getVersion()}")

# first check if the legacy layer is active
print(f"Legacy layer active: {ij.legacy.isActive()}")

Operating in headless mode - the original ImageJ will have limited functionality.


ImageJ2 version: 2.14.0/1.54f
Legacy layer active: True


In [2]:
# demonstrate access to classes
print(ij.IJ)
print(ij.ResultsTable)
print(ij.RoiManager)
print(ij.WindowManager)

Operating in headless mode - the IJ class will not be fully functional.
Operating in headless mode - the ResultsTable class will not be fully functional.
Operating in headless mode - the RoiManager class will not be fully functional.
Operating in headless mode - the WindowManager class will not be fully functional.


<java class 'ij.IJ'>
<java class 'ij.measure.ResultsTable'>
<java class 'ij.plugin.frame.RoiManager'>
<java class 'ij.WindowManager'>


Note the warnings! We're currently in headless mode. The many legacy ImageJ functions operate limitedly or not at all in headless mode. For example the `RoiManager` is not functional in a true headless enviornment.

## 5.2 Register functions to start with ImageJ

Functions can be executed during ImageJ's initialization routine by registering the functions with PyImageJ's callback mechanism `when_imagej_starts()`. This is particularly useful for macOS users in `gui` mode, allowing functions to be called before the Python [REPL/interpreter](https://docs.python.org/3/tutorial/interpreter.html) is [blocked](Initialization.md/#gui-mode).

The following example uses `when_imagej_starts()` callback display a to `uint16` 2D NumPy array it with ImageJ's viewer, print it's dimensions (_i.e._ shape) and open the `RoiManager` while ImageJ initializes.

```python
import imagej
import numpy as np

# register functions
arr = np.random.randint(0, 2**16, size=(256, 256), dtype=np.uint16) # create random 16-bit array
imagej.when_imagej_starts(lambda ij: ij.RoiManager.getRoiManager()) # open the RoiManager
imagej.when_imagej_starts(lambda ij: ij.ui().show(ij.py.to_dataset(arr))) # convert and display the array
imagej.when_imagej_starts(lambda _: print(f"array shape: {arr.shape}"))

# initialize imagej
ij = imagej.init(mode='interactive')
```